### Snappfood Sentiment Analysis 
 ##### بررسی و تحلیل دیتای اسنپ فود و ساخت مدل برای تحلیل کامنت ها و نظرات کاربران

In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import*
import re
import nltk
from nltk.corpus import stopwords

In [8]:
data = pd.read_csv(r'C:\Users\number_one\Desktop\prog\snap food\Snappfood_Sentiment Analysis.csv',delimiter='\t')

In [9]:
data.head()

,Unnamed: 0,comment,label,label_id
0,NaN,واقعا حیف وقت که بنویسم سرویس دهیتون شده افتضاح,SAD,1.0
1,NaN,قرار بود ۱ ساعته برسه ولی نیم ساعت زودتر از مو...,HAPPY,0.0
2,NaN,قیمت این مدل اصلا با کیفیتش سازگاری نداره، فقط...,SAD,1.0
3,NaN,عالللی بود همه چه درست و به اندازه و کیفیت خوب...,HAPPY,0.0
4,NaN,شیرینی وانیلی فقط یک مدل بود.,HAPPY,0.0


In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70000 entries, 0 to 69999
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  520 non-null    object 
 1   comment     70000 non-null  object 
 2   label       70000 non-null  object 
 3   label_id    69480 non-null  float64
dtypes: float64(1), object(3)
memory usage: 2.1+ MB


In [11]:
data.shape

(70000, 4)

In [12]:
data.isnull().sum()

Unnamed: 0    69480
comment           0
label             0
label_id        520
dtype: int64

In [13]:
data = data.drop(columns=['Unnamed: 0'])

In [14]:
data = data.dropna(subset=['comment', 'label_id'])

In [15]:
nltk.download('stopwords')
farsi_stopwords = set([
    'از', 'با', 'که', 'را', 'به', 'برای', 'و', 'یا', 'تا', 'اما', 'اگر', 'همه', 'یک', 'هر', 'این', 'آن', 'او', 'ما', 'شما',
    'آنها', 'می', 'در', 'بر', 'و', 'ها', 'نیز', 'بود', 'شد', 'داشت', 'است', 'هست', 'نیست', 'خواهد', 'شود', 'شود'
])

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\number_one\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [16]:
def clean_text(text):
    text = re.sub(r'\s+' , ' ' ,text)
    text = re.sub(r'[^\w\s]' , '' , text)
    text = text.lower()
    text = ' '.join([word for word in text.split() if word not in farsi_stopwords])
    return text

In [17]:
data['cleaned_comment'] = data['comment'].apply(clean_text)

In [18]:
x = data['cleaned_comment']
y = data['label_id']

In [19]:
x_train , x_test , y_train , y_test = train_test_split(x , y , random_state=42 , test_size=0.3)

In [20]:
vectorizer = TfidfVectorizer(max_features=5000)
x_train_tfidf = vectorizer.fit_transform(x_train)
x_test_tfidf = vectorizer.transform(x_test)

In [21]:
log_moel = LogisticRegression()

log_moel.fit(x_train_tfidf , y_train)

LogisticRegression()

In [22]:
y_pred = log_moel.predict(x_test_tfidf)

In [23]:
print("Logistic Regression Accuracy:", accuracy_score(y_test, y_pred))
print("Logistic Regression Classification Report:")
print(classification_report(y_test, y_pred))

Logistic Regression Accuracy: 0.8544905008635578
Logistic Regression Classification Report:
              precision    recall  f1-score   support

         0.0       0.89      0.81      0.85     10429
         1.0       0.83      0.90      0.86     10415

    accuracy                           0.85     20844
   macro avg       0.86      0.85      0.85     20844
weighted avg       0.86      0.85      0.85     20844



In [24]:
test_comments = x_test  
final_df = pd.DataFrame({
    'Comment': test_comments,         
    'Predicted Label': y_pred         
})



In [25]:
final_df

,Comment,Predicted Label
51551,غذا خیلی سرد صورتیکه فاصله خیلی کم,1.0
23004,بهتره بتونیم ران سینه رو خودمون انتخاب کنیم,0.0
31145,غذا بد حالم خیییییلی بده دل دردو دل پیچه معلوم...,1.0
8071,سلام سابق بسته بندی کیفیت بهتری برخوردار ولی ح...,1.0
28772,سلام خیلی ممنون متشکرم,0.0
...,...,...
24228,خیلی دیر رسید بعد دوبار اعلام تاخیر,1.0
55441,دورو کنجد رو نان معمولی عوض کردن فرستادن بسیار...,1.0
6245,عالی خیلی مرررررسی آب انبه خیلی خیلی خوب,0.0
22357,مزخرفترین پیتزای حال خوردم حیف پول بابت غذا دا...,1.0


In [26]:
final_df.to_csv('snapfood_result.csv', index=False)

این مدل سازی با مدل رگرسیون لاجستیک ساخته شده و دقت 85درصد دارد برای افزایش دقت و پیچیدگی مدل میتوان از شبکه های عصبی یا مدل های پیچیده تر استفاده کرد.